# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [8]:
import torch
import torch.nn as nn
import torch.utils.data as data
from torch.autograd import Variable,no_grad
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [4]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./data/', train=True, transform=transform, download=False)
test_dataset = torchvision.datasets.MNIST(root='./data/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [32]:
class SimpleNet(nn.Module):
    def __init__(self, in_dim, dim1, dim2, out_dim):
        super(SimpleNet, self).__init__()
        self.layer1=nn.Sequential(
            nn.Linear(in_dim, dim1),
            nn.BatchNorm1d(dim1),
            nn.ReLU(True)
        )
        self.layer2=nn.Sequential(
            nn.Linear(dim1, dim2),
            nn.BatchNorm1d(dim2),
            nn.ReLU(True)
        )
        self.layer3=nn.Sequential(
            nn.Linear(dim2, out_dim),
            nn.BatchNorm1d(out_dim),
            nn.ReLU(True)
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x
    
model = SimpleNet(28 * 28, 512, 128, 10)
model = model.cuda()

# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

Next, we can start to train and evaluate!

In [33]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    for images, labels in tqdm(train_loader):
        images = images.view(images.size(0),-1)
        images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        out = model(images)
        loss = criterion(out, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() 


# evaluate
train_acc = 0
test_acc = 0
model.eval()
with no_grad():
    for images, labels in train_loader:
        images = images.view(images.size(0),-1)
        images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        out = model(images)
        _,pred = torch.max(out, 1)
        train_acc += (pred == labels).sum()
 
    for images, labels in test_loader:
        images = images.view(images.size(0), -1)
        images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        out = model(images)
        _, pred = torch.max(out, 1)
        test_acc += torch.sum(pred == labels)




100%|██████████| 468/468 [00:09<00:00, 49.54it/s]


#### Q5:
Please print the training and testing accuracy.

In [ ]:
# evaluate
train_acc = 0
test_acc = 0
model.eval()
with no_grad():
    for images, labels in train_loader:
        images = images.view(images.size(0),-1)
        images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        out = model(images)
        _,pred = torch.max(out, 1)
        train_acc += (pred == labels).sum().item()
 
    for images, labels in test_loader:
        images = images.view(images.size(0), -1)
        images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        out = model(images)
        _, pred = torch.max(out, 1)
        test_acc += torch.sum(pred == labels).item()

In [38]:
print('Training accuracy: %0.2f%%' % (train_acc*100/len(train_dataset)))
print('Testing accuracy: %0.2f%%' % (test_acc*100/len(test_dataset)))

Training accuracy: 99.53%
Testing accuracy: 97.78%
